In [16]:
#! pip install transformers datasets evaluate

In [17]:
from datasets import load_dataset

In [18]:
import pandas as pd

In [19]:
from datasets import load_dataset
dataset_train = load_dataset("csv", data_files="archive/amazon Food Reviews 100k Dataset.csv", 
                       split="train[:10%]")

dataset_eval = load_dataset("csv", data_files="archive/amazon Food Reviews 100k Dataset.csv", 
                       split="train[10%:20%]")

Using custom data configuration default-7291ad35b03e23bd
Found cached dataset csv (/home/jawad/.cache/huggingface/datasets/csv/default-7291ad35b03e23bd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-7291ad35b03e23bd
Found cached dataset csv (/home/jawad/.cache/huggingface/datasets/csv/default-7291ad35b03e23bd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


In [20]:
dataset_train = dataset_train.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

dataset_eval = dataset_eval.rename_column("Review", "text")\
    .rename_column("Rating", "label")\
    .remove_columns(['Id'])

## Tokenizer

In [21]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [22]:
# apply tokenizer
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_eval = dataset_eval.map(tokenize_function, batched=True)

Loading cached processed dataset at /home/jawad/.cache/huggingface/datasets/csv/default-7291ad35b03e23bd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-650eeb2548afead8.arrow
Loading cached processed dataset at /home/jawad/.cache/huggingface/datasets/csv/default-7291ad35b03e23bd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-98d3479d6539a376.arrow


In [23]:
# https://download.pytorch.org/whl/cu116
# !pip3 install torch torchvision torchaudio --extra-index-url 

## Process labels

In [24]:
def minus_1(example):
    example["label"] = example["label"] - 1
    return example

In [25]:
tokenized_dataset_train = tokenized_dataset_train.map(minus_1)
tokenized_dataset_eval = tokenized_dataset_eval.map(minus_1)

Loading cached processed dataset at /home/jawad/.cache/huggingface/datasets/csv/default-7291ad35b03e23bd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-3a53b1310afe160b.arrow
Loading cached processed dataset at /home/jawad/.cache/huggingface/datasets/csv/default-7291ad35b03e23bd/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-86c4d6c36a7c44c5.arrow


In [26]:
tokenized_dataset_train.to_pandas().head()

,label,text,input_ids,token_type_ids,attention_mask
0,4,I have bought several of the Vitality canned d...,"[101, 146, 1138, 3306, 1317, 1104, 1103, 25118...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,0,Product arrived labeled as Jumbo Salted Peanut...,"[101, 22249, 2474, 12893, 1112, 23915, 13685, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,3,This is a confection that has been around a fe...,"[101, 1188, 1110, 170, 14255, 11916, 1988, 111...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,1,If you are looking for the secret ingredient i...,"[101, 1409, 1128, 1132, 1702, 1111, 1103, 3318...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,4,Great taffy at a great price. There was a wid...,"[101, 2038, 27629, 12383, 1120, 170, 1632, 394...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


## Load the model

Start by loading your model and specify the number of expected labels.

In [27]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [28]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [3]:
#model = model.to('cuda')

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

Training hyperparameters - Specify where to save the checkpoints from your training.
specify the evaluation_strategy parameter in your training arguments to report the evaluation metric at the end of each epoch:

In [29]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", 
                                  evaluation_strategy="epoch"
                                 )

## Evalute

In [30]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [31]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Trainer

In [32]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_eval,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/jawad/anaconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 3750
  Number of trainable parameters = 108314117


Epoch,Training Loss,Validation Loss


In [2]:
trainer.evaluate()

NameError: name 'trainer' is not defined

In [1]:
import torch
if torch.cuda.is_available():
    print("Number of GPU devices:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print("Device name:", torch.cuda.get_device_name(i))
else:
    print("No GPU available.")


No GPU available.


In [ ]:
https://www.nvidia.com/Download/index.aspx